[github](https://github.com/seancorfield/next-jdbc)



# next.jdbc

`clojure.java.jdbc`の次の世代。データベースにアクセスするJDBCに対する新しい低レベルなClojureラッパー。

## TL;DR

Clojarsとcljdocの最新版

[![Clojars Project](https://clojars.org/seancorfield/next.jdbc/latest-version.svg)](https://clojars.org/seancorfield/next.jdbc) [![cljdoc badge](https://cljdoc.org/badge/seancorfield/next.jdbc)](https://cljdoc.org/d/seancorfield/next.jdbc/CURRENT)


この文書は1.0.10リリース後のマスターです。
[see the CHANGELOG](CHANGELOG.md).

* [Getting Started](/doc/getting-started.md)
* [Migrating from `clojure.java.jdbc`](/doc/migration-from-clojure-java-jdbc.md)
* Feedback via [issues](https://github.com/seancorfield/next-jdbc/issues) or in the [`#sql` channel on the Clojurians Slack](https://clojurians.slack.com/messages/C1Q164V29/details/) or the [`#sql` stream on the Clojurians Zulip](https://clojurians.zulipchat.com/#narrow/stream/152063-sql).


## Motivation

なぜ別のJDBCライブラリを?
なぜ`clojure.java.jdbc`と異なるAPIを?

* パフォーマンス。`ResultSet`オブジェクトが` clojure.java.jdbc`のハッシュマップのシーケンスに変換される方法には驚くほどのオーバーヘッドがあります。これは、大きな結果セットの場合に顕著になります。そのため、より良い処理方法が必要でした。また、`db-spec`-as-hash-mapに関連するすべての条件付きロジックと解析には、かなりのオーバーヘッドと複雑さがあります。
* 修飾されたキーワードやトランスデューサーなどの使用に基づく最新のAPI：最近の `clojure.java.jdbc`バージョンの`：qualifier`と `reducible-query`はそれに向けたステップでしたが、ライブラリには多くの「レガシー」APIがあります。そして、より集中的で合理化されたAPIを提示して、人々が初日から自然に `IReduceInit` /トランスデューサーアプローチを使用し、修飾キーワードの恩恵を受けるようにします。
* 簡潔さ。`clojure.java.jdbc`はさまざまな方法でSQLを実行しますが、これは矛盾や驚きにつながる可能性があります。`query`、`execute!`、`db-do-commands`はすべて、異なるタイプのSQLを実行する方法です。ステートメントであるため、どちらがどれであるかを覚えておく必要があり、基盤となるJDBC APIの制限に注意する必要があります。

これらが私の3つの主要な動機でした。加えて、 `clojure.java.jdbc`の` db-spec`-as-hash-mapアプローチは、特にサポートされている幅広い矛盾するオプションで、過去に多くのフラストレーションと混乱を引き起こしました。
`next.jdbc`はプロトコルベースであるため、使用方法と直接的なJava JDBCコードを簡単に組み合わせることができます（プロトコルベースのアプローチは全体的なパフォーマンスの向上に貢献します）。
`db-spec`->` DataSource`-> `Connection`のパスがより明確になりました。これにより、人々はより多くの接続を再利用し、アプリのパフォーマンスを向上させることができます。

また、`datafy`/`nav`のサポートをすぐに使用したかったのです（2018年12月に、別の名前空間で文書化されていない実験的なAPIとして`clojure.java.jdbc`に追加されました）。
これは、`execute!`と`execute-one!`のデフォルトの動作です。リダクションで作成する行に`datafy`/`nav`サポートを追加する必要がある場合、プロトコルベースの関数`next.jdbc.result-set/datafiable-row`を`plan`とともに使用できます。

`next.jdbc`がアルファからベータに移行したため、最後の重大な変更が行われ（`reducible!`の名前を`plan`に変更）、APIは安定していると見なされるはずです。今後は、追加的および固定的な変更のみが行われます。

1か月間のアルファビルドのテストが可能になった後、最初のベータビルドが2019年5月24日にリリースされました。6月4日にリリース候補が続き、6月12日に「ゴールド」（1.0.0）リリースがありました。
`next.jdbc`の小さなコアAPIに加えて、`next.jdbc.sql`で利用可能な、`clojure.java.jdbc`のメインAPIに似た"構文糖"(`insert!`、 `query`、`update!`、および`delete!`）があります。違いの詳細については、[`clojure.java.jdbc`からの移行](/doc/migration-from-clojure-java-jdbc.md)を参照してください。



## Usage

`next.jdbc`の背後にある主要な概念は、`javax.sql.DataSource`を作成することから始めることです。
好みのライブラリ（c3p0、hikari-cpなど）を使用して、プールされたデータソースオブジェクトを作成できます。
`next.jdbc`の`get-datasource`関数を使用して、 `db-spec`ハッシュマップまたはJDBC URL（文字列）から`DataSource`を作成できます。
基礎となるプロトコルである`Sourceable`は、より多くのものを`DataSource`に変換できるように拡張できます（また、オブジェクトのメタデータおよび型を介して拡張できます）。

`DataSource`から、あなたまたは`next.jdbc`のいずれかが`get-connection`関数を介して`java.sql.Connection`を作成できます。
あなたは生成された接続を変更するために`get-connection`に対するオプションのハッシュマップを指定できます。`:read-only`、 `:auto-commit`。

`next.jdbc`の主要なSQL実行APIは次のとおりです。
* `plan` -- `IReduceInit`を生成します。これは、reduceされると、SQLステートメントを実行し、可能な限り少ないオーバーヘッドで `ResultSet`を削減します。
* `execute!` -- SQLステートメントを実行し、 `:<table>/<column>`の形式の修飾されたキーワードをカラム名に使用する、実現ハッシュマップのベクトルを生成します。複数のテーブルにまたがって結合する場合、修飾キーワードには各カラムの元のテーブルが反映されます。 SQLが、関連付けられたテーブルに由来しない名前付きの値を生成する場合、単に、非修飾キーワードが使用されます。 `execute!`によって返される実現ハッシュマップは`Datafiable`であり、したがって`Navigable`です（Clojure 1.10の`datafy`および`nav`関数、およびCognitectのREBLなどのツールを参照）。あるいは、 `{:builder-fn rs/as-arrays}`を指定して、カラム名の後にrow値のベクトルが続くベクトルを生成できます。 `:builder-fn`のデフォルトは`rs/as-maps`ですが、非修飾の `:<column>`カラム名が必要な場合は `rs/as-nqualified-maps`と`rs/as-unqualified-arrays`もあります（およびこれらすべての小文字の変形もあります）。
* `execute-one！` -- SQLまたはDDLステートメントを実行し、単一の実現ハッシュマップを生成します。 `execute-one!`によって返される実現ハッシュマップは`Datafiable`であり、したがって`Navigable`です。

さらに、`plan`、`execute!`、または`execute-one!`に渡すことができる`Connection`から`PreparedStatement`（`prepare`）を作成し、トランザクション内でコードを実行するAPI関数があります（ `transact`関数と`with-transaction`マクロ）。

`next.jdbc`は生のJava JDBC型を使用するため、`with-open`を直接使用して接続を再利用し、接続が正しくクリーンアップされるようにすることができます。

```clojure
  (let [my-datasource (jdbc/get-datasource {:dbtype "..." :dbname "..." ...})]
    (with-open [connection (jdbc/get-connection my-datasource)]
      (jdbc/execute! connection [...])
      (reduce my-fn init-value (jdbc/plan connection [...]))
      (jdbc/execute! connection [...])))
```


### Usage scenarios

APIの設計を推進した3つの使用シナリオがあります。
* SQLステートメントを実行し、それを単一の熱心な操作で処理します。これにより、データベースから結果をストリーミングできるようになり（データベース固有の方法でJDBCを説得する方法）、結果を返す前にリソースをクリーンアップします- -削減が`reduced`を介して短絡された場合でも。この使用法は`plan`によってサポートされています。これは最速のアプローチである可能性が高く、SQLクエリで考慮する最初のオプションです。
* SQLまたはDDLステートメントを実行して、 `ResultSet`から最初の行、最初に生成されたキー結果（再び、`ResultSet`から）、または最初のどちらも使用できない場合の結果（`next.jdbc`は、更新カウントのみを返すことができる場合に`{:next.jdbc/update-count N}`を生成します）。この使用法は`execute-one!`でサポートされています。これはおそらく、ほとんどのクエリ以外の操作に最適です。
* SQLステートメントを実行して、完全に実現された`Datafiable`結果セット（ハッシュマップのベクトル）を取得します。この使用法は`execute!`によってサポートされています。カラム名/row値のベクトルを作成することもできます（ `next.jdbc.result-set/as-arrays`）。

さらに、 `clojure.java.jdbc`と同じ名前を使用して、行の挿入、クエリの実行、行の更新、および行の削除を行うための便利な関数（「構文糖」）が提供されます。これらはSQLの作成を伴うため、 `next.jdbc.sql`にあります。これらはコアAPIの一部とは見なされません。

## More Detailed Documentation

* [Getting Started](/doc/getting-started.md)
* [Friendly SQL Functions](/doc/friendly-sql-functions.md)
* [Result Set Builders](/doc/result-set-builders.md)
* [Prepared Statements](/doc/prepared-statements.md)
* [Transactions](/doc/transactions.md)
* [All The Options](/doc/all-the-options.md)
* [`datafy`, `nav`, and `:schema`](/doc/datafy-nav-and-schema.md)
* [Migration from `clojure.java.jdbc`](/doc/migration-from-clojure-java-jdbc.md)

## License

Distributed under the Eclipse Public License version 1.0.

